In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install tqdm
!pip install segmentation_models_pytorch
!pip install albumentations
!pip install wandb

     |████████████████████████████████| 87 kB 7.3 MB/s 
     |████████████████████████████████| 58 kB 8.2 MB/s 
     |████████████████████████████████| 376 kB 48.7 MB/s 
  Created wheel for efficientnet-pytorch: filename=efficientnet_pytorch-0.6.3-py3-none-any.whl size=12421 sha256=cfdb680b61c947ecd3c94cf41abcb9d693c182e94ed361e81c0d34c6dfdf349c
  Stored in directory: /root/.cache/pip/wheels/90/6b/0c/f0ad36d00310e65390b0d4c9218ae6250ac579c92540c9097a
  Created wheel for pretrainedmodels: filename=pretrainedmodels-0.7.4-py3-none-any.whl size=60965 sha256=ef90bd0fb5a8310e8f678a733348c06eb86f1b0394b355941390d06c5fecd7ab
  Stored in directory: /root/.cache/pip/wheels/ed/27/e8/9543d42de2740d3544db96aefef63bda3f2c1761b3334f4873
Successfully built efficientnet-pytorch pretrainedmodels
     |████████████████████████████████| 631 kB 25.7 MB/s 
  Created wheel for imgaug: filename=imgaug-0.2.6-py3-none-any.whl size=654020 sha256=69815e14e2f7dc8a728acced6825e924e030b5468b1ef6cb2e09f43f98b9950e
  

In [ ]:
import os
import glob
import torch
import cv2 as cv
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
from scipy.misc import electrocardiogram
from scipy.signal import find_peaks
from torch.utils.tensorboard import SummaryWriter

from torch.utils.data import Dataset

In [ ]:
cd drive/Shareddrives/KOHI_의료영상1팀/

/content/drive/Shareddrives/KOHI_의료영상1팀


In [ ]:
image_dir = 'Training_Data/image'
mask_dir = 'Training_Data/mask'

## Load Image

load image를 할때 cv.imread로 미리 읽어오면 메모리가 부족할 수도 있는 현상이 발생할 수도 있으므로,

imread는 dataloader의 __getitem__에서 해결

In [ ]:
input_list = os.listdir(image_dir)

In [ ]:
len(input_list)

2324

In [ ]:
dataset = []

for input in tqdm(input_list):
  image = os.path.join(image_dir, input)
  label = os.path.join(mask_dir, input)

  dataset.append({'name': input[:-4], 'image_path':image, 'label_path':label})

100%|██████████| 2324/2324 [00:00<00:00, 141696.16it/s]


### split train/test data

In [ ]:
num_train = int(len(dataset)*0.8)
num_rest = len(dataset)-num_train

if num_rest % 2 == 1:
  num_val = int(num_rest/2)+1
  num_test = int(num_rest/2)
else:
  num_val = int(num_rest/2)
  num_test = int(num_rest/2)

data_train = dataset[:num_train]
data_val = dataset[num_train:num_train+num_val]
data_test = dataset[num_train+num_val:]
print(f'train dataset 개수: {len(data_train)}, valid dataset 개수: {len(data_val)}, test dataset 개수: {len(data_test)}')

lr = 1e-5
batch_size=8
train_continue= False
augment=True
image_size=256

optim_mode = 'adam'

ckpt_dir=f'이승아/checkpoint/U-Net/Dice/{lr}(padding)'
if not os.path.isdir(ckpt_dir):
    os.mkdir(ckpt_dir)

result_dir =f'이승아/result/U-Net/Dice/{lr}(padding)'
if not os.path.isdir(result_dir):
    os.mkdir(result_dir)

image_size = 256

num_epoch = 100

num_batch_train = np.ceil(num_train / batch_size)  
num_batch_train = np.ceil(num_val / batch_size)  
num_batch_test = np.ceil(num_test / batch_size)

train dataset 개수: 1859, valid dataset 개수: 233, test dataset 개수: 232


## Augmentation

In [ ]:
import albumentations as A
from torch.utils.data import DataLoader

In [ ]:
def _normalization(input):
  input = (input - input.min()) / (input.max() - input.min())
  return input

def _standardization(input):
  input = (input - mean) / std
  return input

def _to_tensor(input, label, name):
  input = input.astype('float32')
  label = label.astype('float32')

  input = input.reshape((1, label.shape[0], label.shape[1]))

  label = label.reshape((1, label.shape[0], label.shape[1]))
  
  data = {'name':name, 'input': torch.from_numpy(input), 'label': torch.from_numpy(label)}
  return data

def _random_augment(input, label):
  h, w  = input.shape
  
  transform = A.Compose([A.HorizontalFlip(p=0.5),
                      A.VerticalFlip(p=0.5),
                      A.RandomCrop(height=int(h*0.8), width=int(w*0.8), p=0.5)
                      ]
                      , additional_targets={'label': 'image'})

  augmented = transform(image=input, label=label)

  input = augmented['image']
  label = augmented['label']

  return input, label

## DataLoader

In [ ]:
def resize_image(input, label):
  # shape: (height, width, channel)
  # aspect ratio를 고려하여 resize
  if input.shape[1] > input.shape[0]:   # height를 기준으로
    r = image_size / input.shape[1]
    dim = ( image_size, int(input.shape[0] * r))
  else:                                 # width를 기준으로
    r = image_size / input.shape[0]
    dim = (int(input.shape[1] * r), image_size)
    
  resized_input = cv.resize(input, dim, interpolation = cv.INTER_AREA)
  resized_label = cv.resize(label, dim, interpolation = cv.INTER_NEAREST)
  
  return resized_input, resized_label

def padding_image(input, label):
  input_size = input.shape
  target_size = (image_size, image_size)

  if input_size[1] < image_size:
    padding_range = int(target_size[1]-input_size[1])
  elif input_size[0] < image_size:
    padding_range = int(target_size[0]-input_size[0])
  else:
    return input, label

  if padding_range%2 == 0:
    padding_size = (int(padding_range/2), int(padding_range/2))
  else:
    padding_size = (int(padding_range/2), int(padding_range/2)+1)
  
  if input_size[1] < image_size:
    npad= ((0,0),padding_size)
  elif input_size[0] < image_size:
    npad= (padding_size, (0,0))

  padding_input = np.pad(input, npad,'constant', constant_values=(0))
  padding_label = np.pad(label, npad,'constant', constant_values=(0))

  return padding_input, padding_label

In [ ]:
# DataLoader

row_ratio = 0.15
col_ratio = 0.15

def label_preprocessing(label):
  label = label/255.

  return label

class VertebraeDataset(Dataset):

  def __init__(self, data, augment=False):
    super(VertebraeDataset, self).__init__()

    self.data = data      
    self.augment = augment
    self.image_size = image_size

  def __getitem__(self, index):
    # Read input, label
    name = self.data[index]['name']
    input = cv.imread(self.data[index]['image_path'])
    try:
      input = cv.cvtColor(input, cv.COLOR_BGR2GRAY)
    except:
      print(self.data[index]['image_path'])
      exit(-1)

    label = cv.imread(self.data[index]['label_path'])
    try:
      label = cv.cvtColor(label, cv.COLOR_BGR2GRAY)  
    except:
      print(self.data[index]['label_path'])
      exit(-1)
    label = label/255.

    # # 영상 개선
    clahe = cv.createCLAHE(clipLimit=2.0, tileGridSize=(16, 16))
    input = clahe.apply(input)
    
    input = input / 255.0
    
    if self.augment:
      input, label = _random_augment(input, label)
      
    # Resize and pad Image
    input, label = resize_image(input, label)
    input, label = padding_image(input, label)

    data = _to_tensor(input, label, name)

    return data

  def __len__(self):
    return len(self.data)

In [ ]:
def get_data(data):
    ### Train Dataset 가져오기
    dataset = VertebraeDataset(data, augment=augment)
    loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)  

    return dataset, loader

# pytorch Dataloader
train_dataset, train_loader = get_data(data_train)
val_dataset, val_loader = get_data(data_val)
test_dataset, test_loader = get_data(data_test)

_data = test_dataset.__getitem__(1)

# test_dataset, test_loader = get_data(data_train[:1])

## Parameter 설정


In [ ]:
import segmentation_models_pytorch as smp
from torch import nn
import torch.nn.functional as F
from torch.optim.lr_scheduler import StepLR

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

class DiceLoss(nn.Module):
    """
    Dice score loss function
    """
    def __init__(self):
        super(DiceLoss, self).__init__()
        self.smooth = 1.0

    def forward(self, output, label):
        assert output.size() == label.size()
        output = output[:, 0].contiguous().view(-1)
        label = label[:, 0].contiguous().view(-1)
        intersection = (output * label).sum()
        dsc = (2. * intersection + self.smooth) / (output.sum() + label.sum() + self.smooth)

        return 1. - dsc


class DiceBCELoss(nn.Module):
    def __init__(self, weight=None, size_average=True):
        super(DiceBCELoss, self).__init__()

    def forward(self, inputs, targets, smooth=1):

        # comment out if your model contains a sigmoid or equivalent activation layer
        inputs = torch.sigmoid(inputs)

        # flatten label and prediction tensors
        inputs = inputs.view(-1)
        targets = targets.view(-1)

        intersection = (inputs * targets).sum()
        dice_loss = 1 - (2. * intersection + smooth) /(inputs.sum() + targets.sum() + smooth)
        BCE = F.binary_cross_entropy(inputs, targets, reduction='mean')
        Dice_BCE = BCE + dice_loss

        return Dice_BCE

class Binary_Loss(nn.Module):
    def __init__(self):
        super(Binary_Loss, self).__init__()
        self.criterion = nn.BCEWithLogitsLoss()


    def forward(self, model_output, targets):
        loss = self.criterion(model_output, targets)

       
        return loss

bcedice_loss = DiceBCELoss().cuda()
binary_loss = Binary_Loss().cuda()
dsc_loss = DiceLoss().cuda()

## Build Model

In [ ]:
from segmentation_models_pytorch.unet.model import Unet

In [ ]:
from collections import OrderedDict

## 네트워크 구축하기
class UNet(nn.Module):

    def __init__(self, in_channels=1, out_channels=1, init_features=32):
        super(UNet, self).__init__()

        features = init_features
        self.encoder1 = UNet._block(in_channels, features, name="enc1")
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.encoder2 = UNet._block(features, features * 2, name="enc2")
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.encoder3 = UNet._block(features * 2, features * 4, name="enc3")
        self.pool3 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.encoder4 = UNet._block(features * 4, features * 8, name="enc4")
        self.pool4 = nn.MaxPool2d(kernel_size=2, stride=2)

        self.bottleneck = UNet._block(features * 8, features * 16, name="bottleneck")

        self.upconv4 = nn.ConvTranspose2d(
            features * 16, features * 8, kernel_size=2, stride=2
        )
        self.decoder4 = UNet._block((features * 8) * 2, features * 8, name="dec4")
        self.upconv3 = nn.ConvTranspose2d(
            features * 8, features * 4, kernel_size=2, stride=2
        )
        self.decoder3 = UNet._block((features * 4) * 2, features * 4, name="dec3")
        self.upconv2 = nn.ConvTranspose2d(
            features * 4, features * 2, kernel_size=2, stride=2
        )
        self.decoder2 = UNet._block((features * 2) * 2, features * 2, name="dec2")
        self.upconv1 = nn.ConvTranspose2d(
            features * 2, features, kernel_size=2, stride=2
        )
        self.decoder1 = UNet._block(features * 2, features, name="dec1")

        self.conv = nn.Conv2d(
            in_channels=features, out_channels=out_channels, kernel_size=1
        )

    def forward(self, x):
        enc1 = self.encoder1(x)
        enc2 = self.encoder2(self.pool1(enc1))
        enc3 = self.encoder3(self.pool2(enc2))
        enc4 = self.encoder4(self.pool3(enc3))

        bottleneck = self.bottleneck(self.pool4(enc4))

        dec4 = self.upconv4(bottleneck)
        dec4 = torch.cat((dec4, enc4), dim=1)
        dec4 = self.decoder4(dec4)
        dec3 = self.upconv3(dec4)
        dec3 = torch.cat((dec3, enc3), dim=1)
        dec3 = self.decoder3(dec3)
        dec2 = self.upconv2(dec3)
        dec2 = torch.cat((dec2, enc2), dim=1)
        dec2 = self.decoder2(dec2)
        dec1 = self.upconv1(dec2)
        dec1 = torch.cat((dec1, enc1), dim=1)
        dec1 = self.decoder1(dec1)
        return torch.sigmoid(self.conv(dec1))

    @staticmethod
    def _block(in_channels, features, name):
        return nn.Sequential(
            OrderedDict(
                [
                    (
                        name + "conv1",
                        nn.Conv2d(
                            in_channels=in_channels,
                            out_channels=features,
                            kernel_size=3,
                            padding=1,
                            bias=False,
                        ),
                    ),
                    (name + "norm1", nn.BatchNorm2d(num_features=features)),
                    (name + "relu1", nn.ReLU(inplace=True)),
                    (
                        name + "conv2",
                        nn.Conv2d(
                            in_channels=features,
                            out_channels=features,
                            kernel_size=3,
                            padding=1,
                            bias=False,
                        ),
                    ),
                    (name + "norm2", nn.BatchNorm2d(num_features=features)),
                    (name + "relu2", nn.ReLU(inplace=True)),
                ]
            )
        )

model = UNet().to(device)

# Optimizer Adam 설정
if optim_mode == 'adam':
  optim = torch.optim.Adam(model.parameters(), lr=lr)
elif optim_mode == 'sgd':
  optim = torch.optim.SGD(model.parameters(), lr=lr)
elif optim_mode == 'adamW':
  optim = torch.optim.Adam(model.parameters(), lr=lr)
print(optim_mode)

adam


In [ ]:
from torchsummary import summary

print(summary(model, (1, image_size, image_size)))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 32, 256, 256]             288
       BatchNorm2d-2         [-1, 32, 256, 256]              64
              ReLU-3         [-1, 32, 256, 256]               0
            Conv2d-4         [-1, 32, 256, 256]           9,216
       BatchNorm2d-5         [-1, 32, 256, 256]              64
              ReLU-6         [-1, 32, 256, 256]               0
         MaxPool2d-7         [-1, 32, 128, 128]               0
            Conv2d-8         [-1, 64, 128, 128]          18,432
       BatchNorm2d-9         [-1, 64, 128, 128]             128
             ReLU-10         [-1, 64, 128, 128]               0
           Conv2d-11         [-1, 64, 128, 128]          36,864
      BatchNorm2d-12         [-1, 64, 128, 128]             128
             ReLU-13         [-1, 64, 128, 128]               0
        MaxPool2d-14           [-1, 64,

/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


In [ ]:
fn_tonumpy = lambda x: x.to('cpu').detach().numpy().transpose(0, 2, 3, 1)  # Tensor를 numpy로 변환
fn_denorm = lambda x, mean, std: (x * std) + mean  # DeNomarlization
fn_class = lambda x: 1.0 * (x > 0.4)

In [ ]:
def load_model(ckpt_dir, model, optim):
    if not os.path.exists(ckpt_dir):
        epoch = 0
        return model

    ckpt_lst = os.listdir(ckpt_dir)
    ckpt_lst.sort(key=lambda f: int(''.join(filter(str.isdigit, f))))
    print(os.path.join(ckpt_dir, ckpt_lst[-1]))

    dict_model = torch.load(os.path.join(ckpt_dir, ckpt_lst[-1]))
    model.load_state_dict(dict_model['model'], strict=False)
    optim.load_state_dict(dict_model['optim'])
    epoch = int(ckpt_lst[-1].split('epoch')[1].split('.pth')[0])
    print("Get saved weights successfully.")

    return model, optim, epoch

def save_model(ckpt_dir, model, optim, epoch):
    if not os.path.exists(ckpt_dir):
        os.makedirs(ckpt_dir)

    torch.save({'model': model.state_dict(), 'optim': optim.state_dict()},
                "./%s/model_epoch%d.pth" % (ckpt_dir, epoch))
    print(f'>> save model_{epoch}.pth')

# Train & Valid

In [ ]:
def numeric_score(output, label):
    FP = np.float(np.sum((output == 1) & (label == 0)))
    FN = np.float(np.sum((output == 0) & (label == 1)))
    if FP != 0.0 or FN != 0.0:
        pass
    TP = np.float(np.sum((output == 1) & (label == 1)))
    TN = np.float(np.sum((output == 0) & (label == 0)))

    return FP, FN, TP, TN

def get_score(output, label):
    FP, FN, TP, TN = numeric_score(output, label)
    N = FP + FN + TP + TN

    epsilon = 1e-6

    # Recall : TP / TP+FN
    recall = np.divide(TP, TP + FN + epsilon)
    # Precision : TP / TP+FP
    precision = np.divide(TP, (TP+FP+epsilon))

    accuracy = np.divide((TP + TN), N+epsilon)

    # F1 socre = 2 * (A interect B) / |A| + |B| = 2TP / 2TP + FP + FN
    f1_score = 2 * (precision*recall) / (precision + recall + epsilon)
    dice_coeff = 2*TP / (2*TP+FP+FN+epsilon)

    # J(A,B) = | A intersect B | / | A union B |
    jaccard_score = TP / (TP+FN+FP+ epsilon)

    return recall * 100, precision * 100, accuracy * 100, f1_score*100, jaccard_score*100


In [ ]:
import copy

def metric(gt,pred):
    preds = pred.detach().numpy()
    gts = gt.detach().numpy()

    pred = preds.astype(int)  # float data does not support bit_and and bit_or
    gdth = gts.astype(int)  # float data does not support bit_and and bit_or
    fp_array = copy.deepcopy(pred)  # keep pred unchanged
    fn_array = copy.deepcopy(gdth)
    gdth_sum = np.sum(gdth)
    pred_sum = np.sum(pred)
    intersection = gdth & pred
    union = gdth | pred
    intersection_sum = np.count_nonzero(intersection)
    union_sum = np.count_nonzero(union)

    tp_array = intersection

    tmp = pred - gdth
    fp_array[tmp < 1] = 0

    tmp2 = gdth - pred
    fn_array[tmp2 < 1] = 0

    tn_array = np.ones(gdth.shape) - union

    tp, fp, fn, tn = np.sum(tp_array), np.sum(fp_array), np.sum(fn_array), np.sum(tn_array)

    smooth = 0.001
    precision = tp / (pred_sum + smooth)
    recall = tp / (gdth_sum + smooth)

    false_positive_rate = fp / (fp + tn + smooth)
    false_negtive_rate = fn / (fn + tp + smooth)

    jaccard = intersection_sum / (union_sum + smooth)
    dice = 2 * intersection_sum / (gdth_sum + pred_sum + smooth)
    
    epsilon = 1e-5
    f1_score = 2 * (precision*recall) / (precision + recall + epsilon)

    return false_positive_rate,false_negtive_rate,f1_score
    

In [ ]:
import wandb

In [ ]:
wandb.login()

<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [ ]:
st_epoch = 0

wandb.init(project='scoliosisV2', reinit=True)
wandb.run.name = f'UNet/{lr}/Dice'
wandb.config = {'learning_rate':lr, 'epochs':num_epoch, 'batch_size':batch_size}
wandb.watch(model)

if train_continue:
  model, optim, st_epoch = load_model(ckpt_dir=ckpt_dir, model=model, optim=optim)
else:
  print(f'batch_size:{batch_size}, epoch:{num_epoch}, train_continue:{train_continue}, ckpt_dir:{ckpt_dir}, augment:{augment}')

for epoch in range(st_epoch + 1, num_epoch + 1):
  model.train()  # 모델 학습 모드
  loss_arr = []
  train_f1_score = []


  for iter, data in enumerate(tqdm(train_loader), 1):
    # forward pass
    input = data['input'].to(device)
    label = data['label'].to(device)

    optim.zero_grad()  # 학습 완료 후 gradients 0으로 변경

    output = model(input)

    loss = dsc_loss(output, label)
    loss.backward()
    optim.step()

    _loss = loss.item()  # loss 손실값 불러오기
    loss_arr.append(_loss)
        
    # for metrics
    # logit = torch.sigmoid(output)
    # output = logit.clone()
    output[output>0.5] = 1
    output[output<=0.5] = 0
    
    false_positive_rate,false_negtive_rate,f1_score = metric(label.cpu(), output.cpu())
    train_f1_score.append(f1_score)
        
  wandb.log({
      "f1-score": np.mean(train_f1_score),
      "train-iter_loss": np.mean(loss_arr),})

  wandb.log({"train-loss": np.mean(loss_arr)})
  print("TRAIN: EPOCH %04d / %04d | BATCH %04d / %04d | LOSS %.4f" %
        (epoch, num_epoch, iter, num_batch_train, np.mean(loss_arr)))
  
  #validation
  with torch.no_grad():
    model.eval()
    loss_arr = []
    total_f1_score = []

    for iter, data in enumerate(tqdm(val_loader), 1):
      # forward pass
      input = data['input'].to(device)
      label = data['label'].to(device)

      output = model(input)

      loss = dsc_loss(output, label)
      loss_arr += [loss.item()]
      
      # for metrics
      # logit = torch.sigmoid(output)
      # output = logit.clone()
      output[output>0.5] = 1
      output[output<=0.5] = 0
      
      # save_predictedMask(input, label, output)
  
      false_positive_rate,false_negtive_rate,f1_score = metric(label.cpu(), output.cpu())
      total_f1_score.append(f1_score)
    
    wandb.log({"val-loss": np.mean(loss_arr)})
      
    wandb.log({
        "val/f1-score": np.array(total_f1_score).mean(),
        "val-loss": np.mean(loss_arr)})

  if epoch % 5 == 0:
    save_model(ckpt_dir=ckpt_dir, model=model, optim=optim, epoch=epoch)
    print(f'saved model, {np.mean(loss_arr)}')
  
train_writer.close()

wandb: Currently logged in as: nepersica (use `wandb login --relogin` to force relogin)


batch_size:8, epoch:100, train_continue:False, ckpt_dir:이승아/checkpoint/U-Net/Dice/1e-05(padding), augment:True


100%|██████████| 233/233 [31:03<00:00,  8.00s/it]


TRAIN: EPOCH 0001 / 0100 | BATCH 0233 / 0030 | LOSS 0.7949


100%|██████████| 233/233 [02:46<00:00,  1.40it/s]


...

100%|██████████| 233/233 [02:53<00:00,  1.34it/s]


TRAIN: EPOCH 0100 / 0100 | BATCH 0233 / 0030 | LOSS 0.1407


100%|██████████| 30/30 [00:24<00:00,  1.25it/s]


>> save model_100.pth
saved model, 0.15239123503367105


NameError: ignored